In [1]:
%load_ext autoreload
%autoreload 2

import json
import pandas as pd
import requests

### SIBBR

In [ ]:
import os 

email = os.environ["EMAIL"]
!wget -O sibbr.zip "https://biocache-service.sibbr.gov.br/biocache-service/occurrences/index/download?q=*%3A*&email=${email}&sourceTypeId=&reasonTypeId=3&file=data&extra=dataResourceUid,dataResourceName.p"

In [11]:
!unzip -d sibbr/ sibbr.zip

Archive:  sibbr.zip
  inflating: sibbr/data.csv          
  inflating: sibbr/data_part2.csv    
  inflating: sibbr/data_part3.csv    
  inflating: sibbr/data_part4.csv    
  inflating: sibbr/citation.csv      
  inflating: sibbr/README.html       
  inflating: sibbr/headings.csv      


In [9]:
!ls ./sibbr

citation.csv  data_part2.csv  data_part4.csv  README.html
data.csv      data_part3.csv  headings.csv


### Reflora

In [2]:
family = "Piperaceae"
base_url = "https://servicos.jbrj.gov.br/v2/flora/species/%s" % family

In [3]:
response = requests.get(base_url)
if response.status_code == 200:
    j = response.json()
    if len(j) > 0:
        print("total de especies da %s eh: %d" % (family, len(j)))

total de especies da Piperaceae eh: 1042


In [4]:
if j:
    data = pd.read_json(json.dumps(j))
    data.to_parquet("reflora.parquet", index=False)

/tmp/ipykernel_2251418/3777500992.py:2: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_json(json.dumps(j))


### IBGE

In [5]:
from ibge import *

response = get_data_ibge()
data = [{"id": r["id"], "nome_cidade": r["nome"], "sigla": get_sigla(r), "nome_estado": get_nome_uf(r)} for r in response.json()]
df = pd.json_normalize(data)
df.to_parquet("ibge.parquet", engine="pyarrow", index=False)

In [6]:
!ls

01download.ipynb	  ibge.parquet	 reflora.db.wal
01download-sibbr.ipynb	  ibge.py	 reflora.parquet
02download-reflora.ipynb  plants.db	 sibbr
02limpeza.ipynb		  plants.db.wal  sibbr.db
02load.ipynb		  __pycache__	 sibbr_parquet
03download-ibge.ipynb	  README.md	 sibbr.zip
dados.ipynb		  reflora.db


### FLORA - Lista de Espécies Ameaçadas - 2021.csv

In [6]:
!curl -o flora-ameacada-2021.csv https://dados.mma.gov.br/dataset/41a79b71-445f-4a6a-8c70-d46af991292a/resource/a64f6015-6058-47c5-b4b0-362451240da1/download/flora-ameacada-2021.csv
!ls

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  361k    0  361k    0     0   517k      0 --:--:-- --:--:-- --:--:--  517k
01download.ipynb	     ibge.parquet   README.md
02load.ipynb		     ibge.py	    reflora.parquet
03cleaning.ipynb	     plants.db	    sibbr
flora-ameacada-2021.csv      plants.db.wal  sibbr_parquet
flora-ameacada-2021.parquet  __pycache__    sibbr.zip
